In [1]:
import nltk
import pickle
import argparse
from collections import Counter
import pandas as pd
from tqdm import tqdm
nltk.download('punkt')

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def build_vocab(csv, threshold):
    """Build a simple vocabulary wrapper."""
    data_df = pd.read_csv(csv)
    counter = Counter()
    counter = Counter()
    for i in range(len(data_df)):
        caption = str(data_df.loc[i,'caption'])
        tokens = nltk.tokenize.word_tokenize(caption.lower())

        counter.update(tokens)

        if (i+1) % 10000 == 0:
            print("[{}/{}] Tokenized the captions.".format(i+1, len(data_df)))

    # If the word frequency is less than 'threshold', then the word is discarded.
    words = [word for word, cnt in counter.items() if cnt >= threshold]

    # Create a vocab wrapper and add some special tokens.
    vocab = Vocabulary()
    vocab.add_word('<pad>')
    vocab.add_word('<start>')
    vocab.add_word('<end>')
    vocab.add_word('<unk>')

    # Add the words to the vocabulary.
    for i, word in enumerate(words):
        vocab.add_word(word)
    return vocab

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
vocab=build_vocab('../../data/synth/type/ST_Report.csv', 4)
vocab_path = '../../data/synth/type/ST_vocab.pkl'
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)
print("Total vocabulary size: {}".format(len(vocab)))
print("Saved the vocabulary wrapper to '{}'".format(vocab_path))

Total vocabulary size: 32
Saved the vocabulary wrapper to '../../data/synth/type/ST_vocab.pkl'


In [5]:
vocab.idx2word

{0: '<pad>',
 1: '<start>',
 2: '<end>',
 3: '<unk>',
 4: 'this',
 5: 'image',
 6: 'presents',
 7: 'a',
 8: 'patch',
 9: 'of',
 10: 'gastric',
 11: 'mucosa',
 12: 'affected',
 13: 'by',
 14: 'chronic',
 15: 'gastritis',
 16: '.',
 17: 'the',
 18: 'glands',
 19: 'and',
 20: 'pits',
 21: 'are',
 22: 'elongated',
 23: ',',
 24: 'with',
 25: 'epithelial',
 26: 'cells',
 27: 'showing',
 28: 'mild',
 29: 'dysplasia',
 30: 'characterized',
 31: 'nuclear',
 32: 'enlargement',
 33: 'hyperchromasia',
 34: 'stroma',
 35: 'contains',
 36: 'moderate',
 37: 'lymphoplasmacytic',
 38: 'infiltrate',
 39: 'there',
 40: 'is',
 41: 'no',
 42: 'evidence',
 43: 'intestinal',
 44: 'metaplasia',
 45: 'within',
 46: 'small',
 47: 'segment',
 48: 'depicts',
 49: 'inflammation',
 50: 'areas',
 51: 'normal',
 52: 'has',
 53: 'been',
 54: 'largely',
 55: 'replaced',
 56: 'goblet',
 57: 'absorptive',
 58: 'typical',
 59: 'epithelium',
 60: 'these',
 61: 'metaplastic',
 62: 'arranged',
 63: 'in',
 64: 'well-formed',